In [ ]:
! pip install supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
import requests
from PIL import Image
import numpy as np
import torch
from transformers import AutoProcessor, Owlv2ForObjectDetection
from transformers.utils.constants import OPENAI_CLIP_MEAN, OPENAI_CLIP_STD
import supervision as sv

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import pipeline
# creating our pipeline for OWLVt
checkpoint = "google/owlv2-base-patch16-ensemble"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
# reading the image using opencv2

from PIL import Image
import numpy as np
image_url = "/content/drive/MyDrive/rust/Gemini_Generated_Image_u1vex1u1vex1u1ve.jpg"
# image = '/content/rustbroken.jpg'
image = Image.open(image_url)
image = image.resize((1024,1024))
image_np = np.asarray(image)
Image.fromarray(image_np)

In [ ]:
# finding the rust on image using
predictions = detector(
    image,
    candidate_labels=['rust']
)


In [ ]:
# filtered the bounding boxes based on the scores
filtered_data = [item for item in predictions if item['score'] > 0.15]

In [ ]:
filtered_data

[{'score': 0.40026912093162537,
  'label': 'electric pole and it component',
  'box': {'xmin': 693, 'ymin': 1, 'xmax': 712, 'ymax': 279}},
 {'score': 0.3714740574359894,
  'label': 'electric pole and it component',
  'box': {'xmin': 460, 'ymin': 13, 'xmax': 559, 'ymax': 1020}},
 {'score': 0.3213080167770386,
  'label': 'electric pole and it component',
  'box': {'xmin': 1003, 'ymin': 121, 'xmax': 1012, 'ymax': 233}},
 {'score': 0.3083153963088989,
  'label': 'electric pole and it component',
  'box': {'xmin': 927, 'ymin': 35, 'xmax': 962, 'ymax': 281}},
 {'score': 0.27171629667282104,
  'label': 'electric pole and it component',
  'box': {'xmin': 938, 'ymin': 53, 'xmax': 948, 'ymax': 278}},
 {'score': 0.24012136459350586,
  'label': 'electric pole and it component',
  'box': {'xmin': 606, 'ymin': 104, 'xmax': 612, 'ymax': 166}},
 {'score': 0.23212721943855286,
  'label': 'electric pole and it component',
  'box': {'xmin': 631, 'ymin': 66, 'xmax': 644, 'ymax': 171}},
 {'score': 0.203066

In [ ]:
# used to display the image with bounding box given be above pipeline
from PIL import ImageDraw
image = Image.open(image_url)
image = image.resize((1024,1024))
draw = ImageDraw.Draw(image)


print(len(predictions))
for prediction in predictions:
    box = prediction["box"]
    print(box)
    label = prediction["label"]
    score = prediction["score"]

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="black")

image

To minimze the bounding boxes

In [ ]:
#fn to  Check if box1 is completely contained within box2.
def is_contained(box1, box2):
    """
    Check if box1 is completely contained within box2.
    """
    return (box1['xmin'] >= box2['xmin'] and
            box1['ymin'] >= box2['ymin'] and
            box1['xmax'] <= box2['xmax'] and
            box1['ymax'] <= box2['ymax'])

# to retain only the bigger box
def retain_bigger_boxes(predictions):
    bigger_boxes = []
    for box1 in predictions:
        is_bigger = True
        for box2 in predictions:
          # Check if box1 is completely contained within box2.
            if box1 != box2 and is_contained(box1['box'], box2['box']):
                is_bigger = False
                break
        if is_bigger:
            bigger_boxes.append(box1)
    return bigger_boxes

bigger_boxes = retain_bigger_boxes(predictions)
print(bigger_boxes)





[{'score': 0.29986608028411865, 'label': 'rust', 'box': {'xmin': 434, 'ymin': 764, 'xmax': 475, 'ymax': 827}}, {'score': 0.23346218466758728, 'label': 'rust', 'box': {'xmin': 363, 'ymin': 665, 'xmax': 414, 'ymax': 778}}, {'score': 0.19264453649520874, 'label': 'rust', 'box': {'xmin': 733, 'ymin': 521, 'xmax': 947, 'ymax': 637}}, {'score': 0.17253634333610535, 'label': 'rust', 'box': {'xmin': 253, 'ymin': 439, 'xmax': 280, 'ymax': 485}}, {'score': 0.15675148367881775, 'label': 'rust', 'box': {'xmin': 420, 'ymin': 328, 'xmax': 447, 'ymax': 383}}, {'score': 0.1515590399503708, 'label': 'rust', 'box': {'xmin': 214, 'ymin': 660, 'xmax': 336, 'ymax': 825}}, {'score': 0.15027707815170288, 'label': 'rust', 'box': {'xmin': 170, 'ymin': 771, 'xmax': 198, 'ymax': 804}}, {'score': 0.13758006691932678, 'label': 'rust', 'box': {'xmin': 804, 'ymin': 518, 'xmax': 844, 'ymax': 579}}, {'score': 0.13313159346580505, 'label': 'rust', 'box': {'xmin': 287, 'ymin': 361, 'xmax': 670, 'ymax': 498}}, {'score': 

In [ ]:
# used to display the image with bounding box given be above pipeline with newbounding boxes
from PIL import ImageDraw
image = Image.open(image_url)
image = image.resize((1024,1024))
draw = ImageDraw.Draw(image)


for prediction in bigger_boxes:
    box = prediction["box"]
    print(box)
    label = prediction["label"]
    score = prediction["score"]

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="black")

image

NameError: name 'filtered_data' is not defined

In [ ]:
## filtered the bounding boxes based on the scores
filtered_data = [item for item in bigger_boxes if item['score'] > 0.12]

In [ ]:
filtered_data

[{'score': 0.19175350666046143,
  'label': 'rust only',
  'box': {'xmin': 244, 'ymin': 703, 'xmax': 270, 'ymax': 740}},
 {'score': 0.17603746056556702,
  'label': 'rust only',
  'box': {'xmin': 470, 'ymin': 489, 'xmax': 556, 'ymax': 1028}},
 {'score': 0.1371878683567047,
  'label': 'rust only',
  'box': {'xmin': 212, 'ymin': 709, 'xmax': 229, 'ymax': 766}},
 {'score': 0.13563498854637146,
  'label': 'rust only',
  'box': {'xmin': 539, 'ymin': 816, 'xmax': 814, 'ymax': 1021}},
 {'score': 0.13247919082641602,
  'label': 'rust only',
  'box': {'xmin': 463, 'ymin': 0, 'xmax': 546, 'ymax': 186}},
 {'score': 0.1321040540933609,
  'label': 'rust only',
  'box': {'xmin': 466, 'ymin': -2, 'xmax': 508, 'ymax': 139}},
 {'score': 0.1296740025281906,
  'label': 'rust only',
  'box': {'xmin': 460, 'ymin': 13, 'xmax': 559, 'ymax': 1020}},
 {'score': 0.12865599989891052,
  'label': 'rust only',
  'box': {'xmin': 286, 'ymin': 687, 'xmax': 301, 'ymax': 741}},
 {'score': 0.12829549610614777,
  'label': '

In [ ]:
# used to display the image with bounding box given be above pipeline with new bounding boxes
from PIL import ImageDraw
image = Image.open(image_url)
image = image.resize((1024,1024))
draw = ImageDraw.Draw(image)


print(len(filtered_data))
for prediction in filtered_data:
    box = prediction["box"]
    print(box)
    label = prediction["label"]
    score = prediction["score"]
    print(score)

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="black")

image

In [ ]:
from PIL import Image
# for cropping and saving the image
def crop_and_save(image_path, bbox, output_path):
    # Open the image
    image = Image.open(image_url)
    image = image.resize((1024,1024))

    # Extract bounding box coordinates
    xmin, ymin, xmax, ymax = bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']

    # Crop the image
    cropped_image = image.crop((xmin, ymin, xmax, ymax))

    # Save the cropped image
    cropped_image.resize((1024,1024))
    cropped_image.save(output_path)

# Path to the input image
image_path = 'input_image.jpg'

# Bounding box coordinates
# bbox = {'xmin': 539, 'ymin': 816, 'xmax': 814, 'ymax': 1021}
bbox = {'xmin': 286, 'ymin': 687, 'xmax': 301, 'ymax': 741}

# Output path for the cropped image
output_path = 'cropped_image.jpg'

# Crop and save the image
crop_and_save(image_url, bbox, output_path)


In [ ]:
import os
os.environ['OPENAI_API_KEY'] =  "<your key>"

In [ ]:
import instructor
from instructor import Mode
from openai import OpenAI
from pydantic import BaseModel, Field
from pprint import pprint
import base64
# from dotenv import load_dotenv

# load_dotenv()

client = instructor.patch(OpenAI(), mode=Mode.MD_JSON)
# reading the image and encoding it into encode it into base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

class ObjectDetection(BaseModel):
    # """
    # You are an object detection expert.
    # From the given list ,['High Rust', 'Low Rust', 'Medium Rust'] .
    # First identify if the object exist in the image. If object exist then provide the bounding box of the object and object name.
    # """
    x: int  = Field(description="x coordinate of detected object", default=0)
    y: int = Field(description="y coordinate of detected object", default=0)
    object_found_details: str = Field(description="Details of detected object.", default="")
    image_description: str = Field(descripion="Description of image.", default="")

#fn for calling gpt 4 for an image
def ask_gpt4_vision(system_instrutions, question, image_path):
    base64_image = encode_image(image_path)

    detected = client.chat.completions.create(
        response_model=ObjectDetection,
        model="gpt-4-turbo",
        max_tokens=200,
        messages=[
            {
                "role": "system",
                "content": system_instrutions
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": question},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
    )

    return {"x": detected.x, "y": detected.y, "object_found_details": detected.object_found_details, "image_description": detected.image_description}

system_instructions1 =   """  As an expert in object detection, you specialize in pinpointing specific elements within images.
    Given your expertise, you can efficiently analyze the provided image for any items listed, such as ['High Rust', 'Low Rust', 'Medium Rust', 'No Rust'].
    Rust is formed due to the corrosion on metal surface of electric pole, and it is of red-brownish color. It destroys the metal surface.
    Below is the deifinition of different types of rust:
    Once identified, you will accurately mark their presence by providing both the bounding box coordinates and the name of the detected object.
    You have to identify all the different areas of rust
    It is crucial to exclude any  information from the image to adhere to privacy and data protection standards.
    Output should only contain the type of rust no description is needed.
    """
question = "Can you tell me type of rust on the image."

# question = "Can you tell me type of rust on the image."
# call gpt 4 for cropped image
coordinates = ask_gpt4_vision(system_instructions, question, image_path)


# if bounding boxes are present.
if bounding_boxes !=None:
  bounding_boxes_array = np.array(bounding_boxes)
  annotated_frame_1 = annotate1(image_source=im, boxes=bounding_boxes_array,  phrases=labels)
  annotated_frame = annotated_frame_1[...,::-1]
  rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
  rgb_image1 = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)
  cv2.imwrite('result.jpg', rgb_image1)
else:
  print("Image has no visible sign of rust")





OpenCv approach for detecting the rust,It is a different approach

In [ ]:
import numpy as np
import cv2

# using opencv to detect rust

# read and capturing image
img = cv2.imread("7.jpg", cv2.IMREAD_COLOR)

# Convert the imageFrame in
# BGR(RGB color space) to
# HSV(hue-saturation-value)
# color space
hsvFrame = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Set range for corrosion color and
# define mask
# Define lower and upper bounds for rust color
corr_lower = np.array([0, 60, 100], np.uint8)
corr_upper = np.array([15, 200, 220], np.uint8)

corr_mask = cv2.inRange(hsvFrame, corr_lower, corr_upper)

# Morphological Transform, Dilation
# for each corrosion color and bitwise_and operator
# between imageFrame and mask determines
# to detect only that particular corrosion color
kernal = np.ones((5, 5), "uint8")

# For corrosion color
corr_mask = cv2.dilate(corr_mask, kernal)
res_corr = cv2.bitwise_and(img, img, mask=corr_mask)

# Creating contour to track corrosion color
contours, hierarchy = cv2.findContours(
    corr_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
)

# Initialize list to store bounding box coordinates
bounding_boxes = []

# Merge overlapping bounding boxes
for pic, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 300:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, x + w, y + h))

# Create a single bounding box that encloses all smaller bounding boxes
x1 = min(x for x, _, _, _ in bounding_boxes)
y1 = min(y for _, y, _, _ in bounding_boxes)
x2 = max(x2 for _, _, x2, _ in bounding_boxes)
y2 = max(y2 for _, _, _, y2 in bounding_boxes)

# Draw the enclosing bounding box
cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
cv2.putText(
    img, "Corrosion", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255)
)

# Program Termination
cv2_imshow(cv2.resize(img, (1000, 800)))
cv2.waitKey(0)
cv2.destroyAllWindows()
